# Lab 2 - BCC406

## REDES NEURAIS E APRENDIZAGEM EM PROFUNDIDADE

## Regressão Logística e Rede Neural

### Prof. Eduardo e Prof. Pedro

Objetivos:

- Regressão e Descida do Gradiente

Data da entrega : 26/08 

- Complete o código (marcado com ToDo) e quando requisitado, escreva textos diretamente nos notebooks. Onde tiver *None*, substitua pelo seu código.
- Execute todo notebook e salve tudo em um PDF **nomeado** como "NomeSobrenome-LabX.pdf"
- Envie o PDF via google [FORM](https://forms.gle/MWB1mHjTADbnD6kN8)


# Classificação utilizando frameworks

 - Trabalharemos com um problema de classificação: é um gato ou não é um gato.
 - Utilizaremos um framework: o Tensorflow/Keras.

## Importando os pacotes

Primeiro, vamos executar a célula abaixo para importar todos os pacotes que precisaremos.
- [numpy](www.numpy.org) é o pacote fundamental para a computação científica com Python.
- [h5py](http://www.h5py.org) é um pacote comum para interagir com um conjunto de dados armazenado em um arquivo H5.
- [matplotlib](http://matplotlib.org) é uma biblioteca famosa para plotar gráficos em Python.
- [PIL](http://www.pythonware.com/products/pil/) e [scipy](https://www.scipy.org/) são usados aqui para testar seu modelo.
- [tensorflow](https://www.tensorflow.org/?hl=pt-br) é uma biblioteca famosa para criar e treinar modelos de Deep Learning.
- np.random.seed (1) é usado para manter todas as chamadas de funções aleatórias. 

In [81]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import random

from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.python.keras import Sequential, optimizers
from tensorflow.python.keras.optimizers import gradient_descent_v2
from tensorflow.python.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout

Configurando o matplotlib e a geração de dados aleatórios

In [57]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Configurando o Google Colab

In [58]:
# Para Google Colab: Você vai precisar fazer o upload dos arquivos no seu drive e montá-lo
# não se esqueça de ajustar o path para o seu drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Carregando os dados (10pt)

Coloque os arquivos *train_catvnonvat.h5* e *test_catvnoncat.h5* na pasta raiz do seu Drive. Ambos os arquivos estão na pasta dataset da pasta compartilhada.

In [59]:
# Lendo os dados (gato/não-gato)
def load_dataset():

  train_dataset = h5py.File('drive/MyDrive/train_catvnoncat.h5', "r")
  train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
  train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

  test_dataset = h5py.File('drive/MyDrive/test_catvnoncat.h5', "r")
  test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
  test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

  classes = np.array(test_dataset["list_classes"][:]) # the list of classes
  train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
  test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
  
  return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [60]:
# Lendo os dados (gato/não-gato)
treino_x_orig, treino_y, teste_x_orig, teste_y, classes = load_dataset()

### Pré-processamento dos dados

Pre-processamento necessário. Iremos converter a imagem 3D (64x64x3) em um único vetor 1D (12288 = 64x64x3). A figura mostra um exemplo do pré-processamento executado (imagem vetorizada)

![Arq,widht=10](https://drive.google.com/uc?export=view&id=1zCnEB2rwc4lXU_7RTS4TXhqCwsJubg7H)

<caption><center> <u>Figura</u>: Vetorização de uma imagem. <br> </center></caption>

### **ToDo:** Vetorização da imagem (10pt)

Converta as imagens do formato 64x64x3 para 12288x1.

In [61]:
m_treino = len(treino_x_orig)
m_teste = len(teste_x_orig)
num_px = teste_x_orig[1].shape[1]

#  Vetorizando as imagens de treinamento e teste 

### Início do código ###
treino_x = treino_x_orig.reshape(m_treino, num_px*num_px*3) # dica : utilize reshape para mudar o formato dos dados
### Fim do código ###

### Início do código ###
# Normalize os dados para ter valores de recurso entre 0 e 1.
teste_x = teste_x_orig.reshape(m_teste, num_px*num_px*3) / 255 # dica : utilize reshape para mudar o formato dos dados
### Fim do código ###


## Testando redes neurais e *sigmoid*

Para classificação de classes 0 ou 1, pode-se ter um único neurônio de saída e deve-se usar a operação sigmoid antes de se calcular o custo (mean-squared error ou binary cross entropy).



### Testando uma rede com uma camada oculta

![Arq,widht=10](https://drive.google.com/uc?export=view&id=19mux_FFpeZkj5YiV51bNE2CK3i2nBSau)
<caption><center> <u>Figura 7</u>: Rede neural com 2 camadas. <br> Resumo do modelo: ***ENTRADA -> LINEAR -> RELU -> LINEAR -> SIGMOID -> SAIDA***. </center></caption>

<!--
<u>Detailed Architecture of figure 2</u>:
- The input is a (64,64,3) image which is flattened to a vector of size $(12288,1)$. 
- The corresponding vector: $[x_0,x_1,...,x_{12287}]^T$ is then multiplied by the weight matrix $W^{[1]}$ of size $(n^{[1]}, 12288)$.
- You then add a bias term and take its relu to get the following vector: $[a_0^{[1]}, a_1^{[1]},..., a_{n^{[1]}-1}^{[1]}]^T$.
- You then repeat the same process.
- You multiply the resulting vector by $W^{[2]}$ and add your intercept (bias). 
- Finally, you take the sigmoid of the result. If it is greater than 0.5, you classify it to be a cat.
!-->

In [62]:
## Setando a seed
np.random.seed(1)
random.seed(1)

### Executar uma rede de 1 camada oculta ###
# Camadas da rede = [12288, 200, 1] 

# Definição do modelo
model = Sequential()
model.add(Dense(200, input_shape=(12288,), activation='relu', name='CamadaOculta'))
model.add(Dense(1, activation='sigmoid', name='CamadaClassificacao'))

# Compilando o modelo
model.compile(loss='binary_crossentropy', optimizer='adam')

# Imprimindo a arquitetura da rede proposta
model.summary()

# Treinando o modelo
model.fit(treino_x, treino_y.reshape(-1), epochs=100)

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
CamadaOculta (Dense)         (None, 200)               2457800   
_________________________________________________________________
CamadaClassificacao (Dense)  (None, 1)                 201       
Total params: 2,458,001
Trainable params: 2,458,001
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
7/7 [==============================] - 1s 17ms/step - loss: 1434.7129
Epoch 2/100
7/7 [==============================] - 0s 18ms/step - loss: 205.3096
Epoch 3/100
7/7 [==============================] - 0s 16ms/step - loss: 75.4560
Epoch 4/100
7/7 [==============================] - 0s 16ms/step - loss: 40.7295
Epoch 5/100
7/7 [==============================] - 0s 29ms/step - loss: 25.0947
Epoch 6/100
7/7 [==============================] - 0s 31ms/step - loss: 17.7462
Epoch 7/100
7/7 [=====

**Use os parâmetros treinados** para classificar as imagens de treinamento e teste e verificar a acurácia. 

In [63]:
## Predição da rede
print(f'Acurácia no treino: {accuracy_score(treino_y.reshape(-1), np.round(model.predict(treino_x))) * 100:.2f}')
print(f'Acurácia no teste: {accuracy_score(teste_y.reshape(-1), np.round(model.predict(teste_x))) * 100:.2f}')

Acurácia no treino: 100.00
Acurácia no teste: 64.00


**Resultado esperado:**

```
  Acurácia treino = 100%
  Acurácia teste = 64%
```

#### **ToDo:** Análise dos resultados (5pt)

Por que você obteve 100% no treino e apenas 64% no teste?

`Pode ter ocorrido um caso clássico de overfitting. Provavelmente, o modelo se ajustou bem aos dados de treino, mas não generalizou muito bem a classificação.`

### Testando com uma rede com três camadas ocultas

![Arq,widht=10](https://drive.google.com/uc?export=view&id=19h9LuWkWLVMYgAAoQKTfjJ-Er-tlw8En)
<caption><center> <u>Figura 8</u>: Rede neural com L camadas. <br> Resumo do modelo: ***ENTRADA -> LINEAR -> RELU -> LINEAR -> SIGMOID -> SAIDA***. </center></caption>

In [64]:
## Setando a seed
np.random.seed(1)
random.seed(1)

### Executar uma rede de 3 camadas ocultas ###
# Camadas da rede = [12288, 200, 70, 5, 1] 

# Definição do modelo
model = Sequential()
model.add(Dense(200, input_shape=(12288,), activation='relu', name='CamadaOculta1'))
model.add(Dense(1, activation='sigmoid', name='CamadaClassificacao'))

# Compilando o modelo
model.compile(loss='binary_crossentropy', optimizer='adam')

# Imprimindo a arquitetura da rede proposta
model.summary()


# Treinando o modelo
model.fit(treino_x, treino_y.reshape(-1), epochs=50)

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
CamadaOculta1 (Dense)        (None, 200)               2457800   
_________________________________________________________________
CamadaClassificacao (Dense)  (None, 1)                 201       
Total params: 2,458,001
Trainable params: 2,458,001
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
7/7 [==============================] - 1s 18ms/step - loss: 489.4402
Epoch 2/50
7/7 [==============================] - 0s 18ms/step - loss: 228.5860
Epoch 3/50
7/7 [==============================] - 0s 17ms/step - loss: 140.9641
Epoch 4/50
7/7 [==============================] - 0s 17ms/step - loss: 74.0700
Epoch 5/50
7/7 [==============================] - 0s 16ms/step - loss: 113.8625
Epoch 6/50
7/7 [==============================] - 0s 18ms/step - loss: 134.6651
Epoch 7/50
7/7 [==========

**Use os parâmetros treinados** para classificar as imagens de treinamento e teste e verificar a acurácia. 

In [65]:
## Predição da rede
print(f'Train accuracy: {accuracy_score(treino_y.reshape(-1), np.round(model.predict(treino_x))) * 100:.2f}')
print(f'Test accuracy: {accuracy_score(teste_y.reshape(-1), np.round(model.predict(teste_x))) * 100:.2f}')

Train accuracy: 78.95
Test accuracy: 42.00


**Resultado esperado:**

```
  Acurácia treino = 65.55%
  Acurácia teste = 34.00%
```

#### **ToDo:** Análise dos resultados (5pt)

O resultado com três camadas ocultas foi melhor ou pior do que usa somente uma camada? Tente explicar os motivos.


`O resultado foi pior, segundo a taxa de acurácia. Uma possibilidade é que o modelo é complexo demais para o problema, o que dificulta na generalização do modelo e pode exigir mais épocas para convergência.`

### **ToDo:** Teste uma rede (20pt)

Crie uma arquitetura e treine/teste o seu modelo

In [66]:
### Início do código ###
# Camadas da rede = [12288, 200, 70, 5, 1] 

# Definição do modelo
model = Sequential()
model.add(Dense(1000, input_shape=(12288,),  activation='relu', name='CamadaOculta1'))
model.add(Dropout(.5))
model.add(Dense(1, activation='sigmoid', name='CamadaClassificacao'))


# Compilando o modelo
model.compile(loss='binary_crossentropy', optimizer='adam')

# Imprimindo a arquitetura da rede proposta
model.summary()

# Treinando o modelo
model.fit(treino_x, treino_y.reshape(-1), epochs=100)

print(f'Train accuracy: {accuracy_score(treino_y.reshape(-1), np.round(model.predict(treino_x))) * 100:.2f}')
print(f'Test accuracy: {accuracy_score(teste_y.reshape(-1), np.round(model.predict(teste_x))) * 100:.2f}')

### Fim do código ###

Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
CamadaOculta1 (Dense)        (None, 1000)              12289000  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000)              0         
_________________________________________________________________
CamadaClassificacao (Dense)  (None, 1)                 1001      
Total params: 12,290,001
Trainable params: 12,290,001
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
7/7 [==============================] - 56s 73ms/step - loss: 2954.5066
Epoch 2/100
7/7 [==============================] - 0s 71ms/step - loss: 1076.5275
Epoch 3/100
7/7 [==============================] - 1s 74ms/step - loss: 645.9402
Epoch 4/100
7/7 [==============================] - 1s 72ms/step - loss: 388.8314
Epoch 5/100
7/7 [===========================

## Testando redes neurais e *softmax*

Para classificação de múltiplas classes, tem-se um neurônio de saída para cada classe (como ilustrado no exemplo da Figura 1) e deve-se usar a operação Softmax antes de se calcular o custo (entropia cruzada ou cross-entropy como no exemplo anterior). Consute o capítulo [3.6 do livro](http://d2l.ai/chapter_linear-networks/softmax-regression-scratch.html) para entender melhor.  No caso de se usar softmax, deve-se usar a função **one_hot** para transformar a saída em logits. 

A função **one_hot** transforma um escalar em um **hot encoder**, de acordo com o número de classes.

![Arq,widht=10](https://drive.google.com/uc?export=view&id=1WV_4AT49bYcqsp6PB0FoO4p-gASo0bjL)<caption><center> <u>Figura 1</u>: Rede neural dois neurônios de saída. <br> </center></caption>

### Função de **one-hot encoded**

In [67]:
indices = [0, 1, 2]
depth = 3
tf.one_hot(indices, depth)  # output: [3 x 3]

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)>

**Saída esperada** 
```
 [[1., 0., 0.],
  [0., 1., 0.],
  [0., 0., 1.]]
```

#### **ToDo**: Função one-hot encoded (5pt)

O que a função one-hot encoded faz com o vetor na prática?

`Para cada valor possivel dentro do vetor, gera 'n' números binário únicos. Os número binários gerados são caracterizados por possuem 'n' bits e somente um unico bit '1' na sua representação. A estrutura destes bits torna mais eficiente identificar seu respectivo valor a partir do número binário. `

### Função **SoftMax**

A função softmax transforma a saída em uma distribuição de probabilidades. Assim, a soma de todas as saídas dos neurônio da última camada sempre vai ser igual a 1:

$$
softmax(\mathbf{x}) = \frac{1}{\sum_{i=1}^{n}{e^{x_i}}}
\cdot
\begin{bmatrix}
  e^{x_1}\\\\
  e^{x_2}\\\\
  \vdots\\\\
  e^{x_n}
\end{bmatrix}
$$

o gradiente para o custo usando-se a função softmax é trivial de se calcular:

$$dw = softmax(\mathbf{y_{pred}}) - y$$

In [68]:
indices = [-1., 0., 1.]

print(tf.nn.softmax(indices))

tf.Tensor([0.09003057 0.24472848 0.66524094], shape=(3,), dtype=float32)


**Saída esperada**

```
 [0.09003057, 0.24472848, 0.66524094]
```

Perceba que esse código também funciona se você passar um lote (batch) de amostras

In [69]:
# Veja a saída abaixo
X = np.array([[10., 2., -3.],
              [-1., 5., -20.]])
print(tf.nn.softmax(X))

tf.Tensor(
[[9.99662391e-01 3.35349373e-04 2.25956630e-06]
 [2.47262316e-03 9.97527377e-01 1.38536042e-11]], shape=(2, 3), dtype=float64)


#### **ToDo**: Função softmax (5pt)

O que a função softmax faz com o vetor na prática?

`Converte os valores do vetor em uma distribuição de probabilidade, de acordo com a função sigmoid.`

### Função de erro

Em seguida, deve-se computar o erro entre um vetor predito (**Y_pred**) e o vetor de real de rótulos (**Y_true**). para tal, deve-se usar cross entropy loss, ou verossimilhança negativa (negative log likelihood). A função **cross_entropy()** implementa a verossimilhança negativa.

```
tf.keras.losses.CategoricalCrossentropy()
```

Erro de uma predição bem ruim

In [70]:
y_true = [[1, 0, 0]]
y_pred = [[0.12, 4, 10]]

cce = tf.keras.losses.CategoricalCrossentropy()
print(cce(y_true, y_pred).numpy())


4.7678556


Erro de uma boa predição

In [71]:
y_true = [[1, 0, 0]]
y_pred = [[0.97, 0.01, 0.02]]

cce = tf.keras.losses.CategoricalCrossentropy()
print(cce(y_true, y_pred).numpy())


0.030459179


A função de erro também funciona para um lote de dados.

In [72]:
y_true = np.array([[0, 1, 0],
                   [1, 0, 0],
                   [0, 0, 1]])

y_pred = np.array([[0,   1,    0],
                   [.99, 0.01, 0],
                   [0,   0,    1]])

cce = tf.keras.losses.CategoricalCrossentropy()
print(cce(y_true, y_pred).numpy())


0.0033501784782856703


#### **ToDo:** Função de erro (5pt)

Explique a função de erro *Categorical Cross-entropy*.

``

### Testando uma rede com uma camada oculta

Para esta atividade você deve usar uma loss (ou função de perda) baseada em softmax.




In [73]:
## Setando a seed
np.random.seed(1)
random.seed(1)

### Executar uma rede de 1 camada oculta ###
# Camadas da rede = [12288, 200, 1] 

# Definição do modelo
model = Sequential()
model.add(Dense(200, input_shape=(12288,), activation='relu', name='CamadaOculta'))
model.add(Dense(2, activation='softmax', name='CamadaClassificacao'))

# Compilando o modelo
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Imprimindo a arquitetura da rede proposta
model.summary()

# Treinando o modelo
model.fit(treino_x, tf.one_hot(treino_y.reshape(-1), 2), epochs=100)

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
CamadaOculta (Dense)         (None, 200)               2457800   
_________________________________________________________________
CamadaClassificacao (Dense)  (None, 2)                 402       
Total params: 2,458,202
Trainable params: 2,458,202
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
7/7 [==============================] - 0s 17ms/step - loss: 1763.9198
Epoch 2/100
7/7 [==============================] - 0s 18ms/step - loss: 258.0015
Epoch 3/100
7/7 [==============================] - 0s 16ms/step - loss: 40.9121
Epoch 4/100
7/7 [==============================] - 0s 18ms/step - loss: 29.4248
Epoch 5/100
7/7 [==============================] - 0s 18ms/step - loss: 18.4680
Epoch 6/100
7/7 [==============================] - 0s 17ms/step - loss: 10.8982
Epoch 7/100
7/7 [=====

**Use os parâmetros treinados** para classificar as imagens de treinamento e teste e verificar a acurácia. 

In [74]:
## Predição da rede
print(f'Acurácia no treino: {accuracy_score(treino_y.reshape(-1), np.argmax(model.predict(treino_x), axis=1)) * 100:.2f}')
print(f'Acurácia no teste: {accuracy_score(teste_y.reshape(-1), np.argmax(model.predict(teste_x), axis=1)) * 100:.2f}')

Acurácia no treino: 100.00
Acurácia no teste: 70.00


**Resultado esperado:**

```
  Acurácia treino = 100%
  Acurácia teste = 70%
```

### **ToDo:** Testando outras redes (20pt)

Primero, implemente a rede de três camadas ocultas (mesma arquitetura utilizada com sigmoid). Por fim, repita o teste com uma arquitetura projetada por você, de preferência, bem profunda e mais larga.
Plote a curva de custo (epochs vs loss) para cada um dos dois casos. O que você conclui?

In [75]:
### Início do código ###
## Setando a seed
np.random.seed(1)
random.seed(1)

# Definição do modelo
model = Sequential()
model.add(Dense(200, input_shape=(12288,), activation='sigmoid', name='CamadaOculta1'))
model.add(Dense(200, activation='sigmoid', name='CamadaOculta2'))
model.add(Dense(200, activation='sigmoid', name='CamadaOculta3'))
model.add(Dense(2, activation='softmax', name='CamadaClassificacao'))

# Compilando o modelo
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Imprimindo a arquitetura da rede proposta
model.summary()

# Treinando o modelo
model.fit(treino_x, tf.one_hot(treino_y.reshape(-1), 2), epochs=100)

## Predição da rede
print(f'Acurácia no treino: {accuracy_score(treino_y.reshape(-1), np.argmax(model.predict(treino_x), axis=1)) * 100:.2f}')
print(f'Acurácia no teste: {accuracy_score(teste_y.reshape(-1), np.argmax(model.predict(teste_x), axis=1)) * 100:.2f}')
### Fim do código ###

Model: "sequential_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
CamadaOculta1 (Dense)        (None, 200)               2457800   
_________________________________________________________________
CamadaOculta2 (Dense)        (None, 200)               40200     
_________________________________________________________________
CamadaOculta3 (Dense)        (None, 200)               40200     
_________________________________________________________________
CamadaClassificacao (Dense)  (None, 2)                 402       
Total params: 2,538,602
Trainable params: 2,538,602
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
7/7 [==============================] - 1s 17ms/step - loss: 0.6634
Epoch 2/100
7/7 [==============================] - 0s 19ms/step - loss: 0.6531
Epoch 3/100
7/7 [==============================] - 0s 18ms/step - loss: 0.6460
Epoc

In [83]:
### Início do código ###
## Setando a seed
np.random.seed(1)
random.seed(1)

# Definição do modelo
model = Sequential()
model.add(Dense(500, input_shape=(12288,), activation='relu', name='CamadaOculta1'))
model.add(Dense(100, activation='relu', name='CamadaOculta2'))
model.add(Dense(200, activation='relu', name='CamadaOculta3'))
model.add(Dense(2, activation='softmax', name='CamadaClassificacao'))

# Compilando o modelo
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Imprimindo a arquitetura da rede proposta
model.summary()

# Treinando o modelo
model.fit(treino_x, tf.one_hot(treino_y.reshape(-1), 2), epochs=100)
## Predição da rede
print(f'Acurácia no treino: {accuracy_score(treino_y.reshape(-1), np.argmax(model.predict(treino_x), axis=1)) * 100:.2f}')
print(f'Acurácia no teste: {accuracy_score(teste_y.reshape(-1), np.argmax(model.predict(teste_x), axis=1)) * 100:.2f}')
### Fim do código ###

Model: "sequential_37"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
CamadaOculta1 (Dense)        (None, 500)               6144500   
_________________________________________________________________
CamadaOculta2 (Dense)        (None, 100)               50100     
_________________________________________________________________
CamadaOculta3 (Dense)        (None, 200)               20200     
_________________________________________________________________
CamadaClassificacao (Dense)  (None, 2)                 402       
Total params: 6,215,202
Trainable params: 6,215,202
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
7/7 [==============================] - 1s 39ms/step - loss: 959.4437
Epoch 2/100
7/7 [==============================] - 0s 39ms/step - loss: 212.4634
Epoch 3/100
7/7 [==============================] - 0s 40ms/step - loss: 197.474

### **ToDo:** Variando alguns hiperparâmetros (20pt)

Usando o framework do tensorflow/keras, altere os hiperparâmetros e veja o impacto (gere pelo menos dois novos modelos):

- learning rate,
- Algoritmo de otimização (SGD com momento, ADAM, ADADELTA, RMSPROP),
- inicialização dos pesos: inicialiação aleatória vs uniforme,
- Funções de ativação : troque a sigmoid por (ReLU, GELU, Leaky RELU),

In [86]:
### Início do código ###
## Setando a seed
np.random.seed(1)
random.seed(1)

# Definição do modelo
model = Sequential()
model.add(Dense(200, input_shape=(12288,), activation='relu', name='CamadaOculta1'))
model.add(Dense(100, activation='gelu', name='CamadaOculta2'))
model.add(Dense(200, activation='gelu', name='CamadaOculta3'))
model.add(Dense(2, activation='softmax', name='CamadaClassificacao'))

# Compilando o modelo
model.compile(loss='categorical_crossentropy', optimizer="ADADELTA")

# Imprimindo a arquitetura da rede proposta
model.summary()

# Treinando o modelo
model.fit(treino_x, tf.one_hot(treino_y.reshape(-1), 2), epochs=100)


## Predição da rede
print(f'Acurácia no treino: {accuracy_score(treino_y.reshape(-1), np.argmax(model.predict(treino_x), axis=1)) * 100:.2f}')
print(f'Acurácia no teste: {accuracy_score(teste_y.reshape(-1), np.argmax(model.predict(teste_x), axis=1)) * 100:.2f}')
### Fim do código ###

Model: "sequential_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
CamadaOculta1 (Dense)        (None, 200)               2457800   
_________________________________________________________________
CamadaOculta2 (Dense)        (None, 100)               20100     
_________________________________________________________________
CamadaOculta3 (Dense)        (None, 200)               20200     
_________________________________________________________________
CamadaClassificacao (Dense)  (None, 2)                 402       
Total params: 2,498,502
Trainable params: 2,498,502
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
7/7 [==============================] - 1s 20ms/step - loss: 30.2774
Epoch 2/100
7/7 [==============================] - 0s 22ms/step - loss: 21.0016
Epoch 3/100
7/7 [==============================] - 0s 22ms/step - loss: 18.5134
E

#### **ToDo:** Analisando redes treinadas (5pt)

Qual combinação rendeu o melhor resultado? Tente explicar o por que.

`A rede com o menor número de camadas obteve o melhor resultado. Talvez poe que os outros modelos eram complexo demais para o problema.`